In [1]:
# pip install opencv-python


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import requests
from tqdm.auto import tqdm
import urllib
import cv2
import numpy as np
import threading
import math

In [3]:
host = "192.168.24.160:8082"
token = "e764506728e98e51b5659d386757ca83ca8744c5"
projects = {
    7: 'healed',
    6: 'healing',
    5: 'disease'
    
}
base_dir = 'dataset'
os.makedirs(base_dir, exist_ok=True)

In [10]:
annotations = {}
for project_id, project_name in projects.items():
    data = requests.get(f"http://{host}/api/projects/{project_id}/export?exportType=JSON", 
                 headers={
                     'Authorization': f"Token {token}"
                 }
    )
    annotation = data.json()
    [elm.update({'class': project_name}) for elm in annotation]
    annotations[project_name] = annotation
    with open(f'dataset/annotations-{project_name}.json', 'w') as f:
        json.dump(annotations, f)
    print(f"Saved annotations {project_name} => {len(annotation)}")

Saved annotations healed => 446
Saved annotations healing => 350
Saved annotations disease => 350


In [11]:
datasets = []
for dataset_name, dataset in annotations.items():
    datasets += dataset
with open(f'dataset/annotations.json', 'w') as f:
        json.dump(datasets, f)
print(f"number of dataset : {len(datasets)}")

number of dataset : 1146


In [6]:
def splitList(data, num, num_threads=8):
    window = len(data)/num_threads
    start = math.ceil(num * window)
    stop = math.ceil((num+1) * window)
    return data[start:stop]

def get_image(thread_no, dataset):
    for item in tqdm(dataset):
        data_path = item.get('data', {}).get('image')
        file_name = data_path.split('/')[-1]

        req = urllib.request.Request(f'http://{host}{data_path}')
        req.add_header('Authorization', f"Token {token}")
        req = urllib.request.urlopen(req)
        
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        frame = cv2.imdecode(arr, -1)
        
        image_path = os.path.join(base_dir, 'images', file_name)
        cv2.imwrite(image_path, frame)

In [7]:
NUM_THREAD = 12
os.makedirs(os.path.join(base_dir, 'images'), exist_ok=True)
    
threads=[]
for i in range(NUM_THREAD):
    print(f"Main    : create and start thread {i}.")
    blukData = splitList(datasets, i, num_threads=NUM_THREAD)
    thread = threading.Thread(target=get_image, args=(i,blukData, ) )
    threads.append(thread)
    thread.start()
    
for index, thread in enumerate(threads):
    print(f"Main    : before joining thread {index}.")
    thread.join()
    print(f"Main    : thread {index} done")

Main    : create and start thread 0.
Main    : create and start thread 1.
Main    : create and start thread 2.
Main    : create and start thread 3.
Main    : create and start thread 4.
Main    : create and start thread 5.
Main    : create and start thread 6.
Main    : create and start thread 7.
Main    : create and start thread 8.
Main    : create and start thread 9.
Main    : create and start thread 10.
Main    : create and start thread 11.
Main    : before joining thread 0.


  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

...

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

Main    : thread 0 done
Main    : before joining thread 1.
Main    : thread 1 done
Main    : before joining thread 2.
Main    : thread 2 done
Main    : before joining thread 3.
Main    : thread 3 done
Main    : before joining thread 4.
Main    : thread 4 done
Main    : before joining thread 5.
Main    : thread 5 done
Main    : before joining thread 6.
Main    : thread 6 done
Main    : before joining thread 7.
Main    : thread 7 done
Main    : before joining thread 8.
Main    : thread 8 done
Main    : before joining thread 9.
Main    : thread 9 done
Main    : before joining thread 10.
Main    : thread 10 done
Main    : before joining thread 11.
Main    : thread 11 done
